In [1]:
import os

# import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
# import get_related_artists as from_echo
import re
import pickle
from IPython.display import display, HTML
import python_utils as utils


In [2]:
# Load metadata and features.
tracks = pd.read_csv('fma_metadata/tracks.csv')
r_tracks = pd.read_csv('fma_metadata/raw_tracks.csv')
genres = pd.read_csv('fma_metadata/genres.csv')
features = pd.read_csv('fma_metadata/features.csv')
echonest = pd.read_csv('fma_metadata/echonest.csv')
r_artists = pd.read_csv('fma_metadata/raw_artists.csv')

tracks.shape, genres.shape, features.shape, echonest.shape

/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/anirudhmani/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170

((106576, 53), (163, 5), (106577, 519), (13132, 250))

In [3]:
tracks_sm = tracks.loc[(tracks['set.1'] == 'small') & (tracks['set'] == 'training')]

In [4]:
# Spotify  Business

import sys
import spotipy
import pprint
''' shows recommendations for the given artist
'''


from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='ba6790bdcd434f06b7b577e344c6e0ae', client_secret='145715e565ff48469c306484896a34f5')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None


def get_related(uri):
    
    related = sp.artist_related_artists(uri)
    ans = [] 
    
    for artist in related['artists']:
        ans.append(artist['name'])
#        print artist['name']
        
    
    return ans
    
def save_spot_rel_artists(name, table):
    with open('%s.pickle'%name, 'wb') as writer:
        pickle.dump(table, writer, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_spot_rel_artists(name):
    with open(name, 'rb') as handle:
        thing = pickle.load(handle)   
    return thing

In [5]:
# get spotify artist_id for every artist, and related artists
na = 0
artist_rel = {}
artists_sm = tracks_sm['artist.12'].unique()

for i, elem in enumerate(artists_sm):
    
    artist = re.split(',', elem)[0]
    if i%100 == 0:
        print i
    
#     a_id = from_echo.get_artist(artist)
    try:
        a_id = get_artist(artist)

        if a_id:
    #         related_artists = from_echo.get_related(a_id['uri'])
            related_artists = get_related(a_id['uri'])
    #        print related_artists
            if related_artists:
                artist_rel[artist] = related_artists
    #             artist_rel[artist] = a_id
            else:
                na += 1
        else:
            na += 1
    except:
        continue
        



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800


In [6]:
save_spot_rel_artists("artist_rel_small_training", artist_rel) 

In [5]:
# save_spot_rel_artists("artist_rel_medium_training", artist_rel)
artist_rel = load_spot_rel_artists("artist_rel_small_training.pickle")


In [6]:
len(artist_rel)

1026

In [7]:
# Subsetting Business, Creating Master tables

# Creating Condensed Artist Map, and Master 2D Table
artist_all = {}

for key in artist_rel:
    artist_all[key] = set()

for key in artist_rel:
    for elem in artist_rel[key]:
        if artist_rel.get(elem, None) is not None:
            artist_all[key].add(elem)
            artist_all[elem].add(key)

# assert len(artist_all) == len(artist_rel)  
# deleting artists with no related artist in set
keys = artist_all.keys()
for key in keys:
    if not artist_all[key]:
        del artist_all[key]
#         pass
    else:
        artist_all[key] = list(artist_all[key])
        
# Artist Index Map
artist_index = {}
for i, a in enumerate(artist_all.keys()):
    artist_index[a] = i


In [8]:
artist_rel_master = np.zeros((len(artist_all.keys()), len(artist_all.keys())))

# Master table of artist relation
for i, a in enumerate(artist_all.keys()):
    # going through each artist
    for b in artist_all[a]:
        artist_rel_master[artist_index[a]][artist_index[b]] = 1

# print artist_rel_master[artist_index['Robo']][artist_index['Robo']]

In [9]:
# Master table of artists per genre

# songs per genre
genres_all = {} # maybe redundant
c = 0
artists_set = list(tracks_sm['artist.12'])
genres_artist = {}
i = 0
for gens in tracks_sm['track.9']:
    l = [re.split(']', s) for s in re.split(', |\[', gens)]
    gens = [item for sublist in l for item in sublist]
    for g in gens:
        if g.isdigit():
            if genres_all.get(int(g), -1) != -1:
                pass
            else:
                genres_all[int(g)] = c
                c += 1
            if genres_artist.get(int(g), -1) == -1:
                genres_artist[int(g)] = set()
            if artist_all.get(artists_set[i], None) is not None:
                genres_artist[int(g)].add(artists_set[i]) 
    i += 1



In [10]:
# creating master table of artists per genre

genre_artist_master = np.zeros((len(genres_all.keys()), len(artist_all.keys())))

for g in genres_all.keys():
#     print genres_artist[g]
    for a in genres_artist[g]:
#         print a
        genre_artist_master[genres_all[g]][artist_index[a]] = 1
        
# print genre_artist_master

In [11]:
# Number of songs per artist
songs_per_artist = {}
for k in artist_all.keys():
    songs_per_artist[k] = len(tracks_sm[tracks_sm['artist.12'] == k])

In [12]:
# Calculating D

def calc_D(songs_per_artist, num_artists_set):
    # total number of songs in Set
    num_songs_set = 0
    data = []
    for k in songs_per_artist:
        num_songs_set += songs_per_artist[k]
        if songs_per_artist[k] != 0:
            data.append(songs_per_artist[k])

    data = np.array(data)
    mn = np.mean(data, axis=0)
    sd = np.std(data, axis=0)
    
    D = 1 - (sd/(mn * (np.sqrt(num_artists_set) - 1)))

    return D


In [13]:
# Calculating C

def calc_C(num_artists_set, genres_artist):
    # total number of genres in Set
    sum_gen = genres_artist.sum(axis=1)
    num_gen_set = np.count_nonzero(sum_gen)

    mn = np.mean(sum_gen, axis=0)
    sd = np.std(sum_gen, axis=0)

    C = 1 - (sd/(mn * (np.sqrt(len(sum_gen)) - 1)))

    return C

In [14]:
# Calculating S

def calc_S(artists_in_set, songs_per_artist):
     
    num_edges = np.count_nonzero(artists_in_set)/2

    # total number of artists in Set
    num_artists_set = len(songs_per_artist.keys())

    # std dev of artist connections per artist
    a_edges = artists_in_set.sum(axis=1)
    mn = np.mean(a_edges, axis=0)
    sd = np.std(a_edges, axis=0)

    S = 1 - (sd/(mn * (np.sqrt(num_artists_set) - 1)))
    return S

In [19]:
import copy
    # N is number of Artists in Set

N = 175    
artists_in_set = np.random.choice(artist_all.keys(), N, replace=False)
artists_not_in_set = artist_all.keys()
for a in artists_in_set:
    artists_not_in_set.remove(a)

print len(artists_in_set), len(artists_not_in_set)

# songs per artist subset    
s_p_a = {}
for a in artists_in_set:
    s_p_a[a] = songs_per_artist[a]

# artists per genre subset
num_genres, num_artists = genre_artist_master.shape
g_a = np.zeros((num_genres, N))
a_rel = np.zeros((N, N))
index = 0

ai = [artist_index[k] for k in artists_in_set]

for i in range(num_genres):
    g_a[i, :] = genre_artist_master[i, ai]

for a in artists_in_set:
    ind = artist_index[a]
    a_rel[index, :] = artist_rel_master[ind, ai]
    a_rel[:, index] = artist_rel_master[ai, ind]    
    index +=1

175 219


In [20]:
best_artist = []

C = calc_C(N, g_a)
D = calc_D(s_p_a, N)
S = calc_S(a_rel, s_p_a)
goodness = C * D * S

print goodness

g_list = []
best_set = []
g_list_local = []
g_list_local.append(goodness)

for epoch in range(3):


    # main loop for optimization
    for i, a in enumerate(artists_in_set):
        #remove an artist
        s_p_a[a] = 0
        g_a[:, i] = 0
        a_rel[i, :] = 0
        a_rel[:, i] = 0

        change = 0
        local_goodness = - float('inf')
        # add artist from other set one at a time
        for b in artists_not_in_set:

            # add from other set
            s_p_a[b] = songs_per_artist[b]
            ai[i] = artist_index[b]
            g_a[:, i] = genre_artist_master[:, artist_index[b]]
            a_rel[i, :] = artist_rel_master[artist_index[b], ai]
            a_rel[:, i] = artist_rel_master[ai, artist_index[b]]

            C = calc_C(N, g_a)
            D = calc_D(s_p_a, N)
            S = calc_S(a_rel, s_p_a)

            G = C * D * S
            if G > local_goodness:
                local_goodness = G
                change = b

            # delete other element
            s_p_a[b] = 0
            g_a[:, i] = 0
            a_rel[i, :] = 0
            a_rel[:, i] = 0       

#         print goodness
        if local_goodness > goodness:
            goodness = local_goodness
            ai[i] = artist_index[change]
            s_p_a[change] = songs_per_artist[change]
            g_a[:, i] = genre_artist_master[:, artist_index[change]]
            a_rel[i, :] = artist_rel_master[artist_index[change], ai]
            a_rel[:, i] = artist_rel_master[ai, artist_index[change]]
            artists_not_in_set.append(a)
            artists_not_in_set.remove(change)
        else:
            ai[i] = artist_index[a]
            s_p_a[a] = songs_per_artist[a]
            g_a[:, i] = genre_artist_master[:, artist_index[a]]
            a_rel[i, :] = artist_rel_master[artist_index[a], ai]
            a_rel[:, i] = artist_rel_master[ai, artist_index[a]]



        g_list_local.append(goodness)


    g_list.append(np.mean(np.array(g_list_local), axis=0))
    print epoch, g_list_local[-1]


0.665435019443
0 0.774909637091
1 0.774909637091
2 0.774909637091
